In [1]:
from function import *
from data_preparation import *
from evaluation import *

from openpyxl import Workbook
import pandas as pd
import numpy as np

In [15]:
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem
from rdkit import Chem


def transform_data(input_file_name, output_file_name, updated_binary_label):
    suppl = Chem.SDMolSupplier(input_file_name)
    output_file = open(output_file_name, 'w')
    good = 0
    bad = 0
    i = 0
    unique = set()
    print >> output_file, "molecule ID(RegID),library,existing SMILES,generated SMILES,1024_fingerprint,Pria_SSB_%INH,true_label"
    for mol in suppl:
        i += 1
        if mol is None:
            bad += 1
            continue
        name = mol.GetProp("RegID")
        if name in unique:
            continue
        unique.add(name)
        good += 1
        library = mol.GetProp("Library")
        existing_smiles = mol.GetProp("SMILES")
        generated_smiles = Chem.CanonSmiles(Chem.MolToSmiles(mol))
        generated_mol = Chem.MolFromSmiles(existing_smiles)
        fingerprints = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)

        pria = mol.GetProp("Pria_SSB_%INH")
        pria = float(pria)

        true_label = updated_binary_label[updated_binary_label['Molecule'] == name]['Keck_Pria_AS_Retest']
        true_label = true_label.tolist()[0]
        output_line = "{},{},{},{},{},{},{}".format(name,
                                                    library,
                                                    existing_smiles,
                                                    generated_smiles,
                                                    fingerprints.ToBitString(),
                                                    pria,
                                                    true_label)
        print >> output_file, output_line
    print("{} items in all, {} are good, {} are bad".format(i, good, bad))
    return


if __name__ == '__main__':
    discrete_file = pd.ExcelFile('../dataset/data_preprocessing/screening_smsf_actives_2017_03_10.xlsx')
    Keck_Pria_Retest = discrete_file.parse('Keck_Pria_Retest')
    Keck_Pria_FP = discrete_file.parse('Keck_Pria_FP')
    Keck_RMI = discrete_file.parse('Keck_RMI')
    Xing_MTDH_Retest = discrete_file.parse('Xing_MTDH_Retest')
    Xing_MTDH_DR = discrete_file.parse('Xing_MTDH_DR')
    
    continuous_file = pd.ExcelFile('../dataset/data_preprocessing/screening_smsf_continuous_2017_03_10.xlsx')
    Keck_Pria_Primary = continuous_file.parse('Keck_Pria_Primary')
    Keck_RMI_cdd = continuous_file.parse('Keck_RMI_cdd')
    Xing_MTDH_cdd = continuous_file.parse('Xing_MTDH_cdd')
    
#     transform_data('../dataset/lc123_keckdata.sdf', '../dataset/keck_complete.csv', updated_binary_label)

In [16]:
print Keck_Pria_Retest.dtypes
print
print Keck_Pria_FP.dtypes
print
print Keck_RMI.dtypes
print
print Xing_MTDH_Retest.dtypes
print
print Xing_MTDH_DR.dtypes

Molecule               object
Keck_Pria_AS_Retest     int64
dtype: object

Molecule             object
Keck_Pria_FP_data     int64
dtype: object

Molecule        object
Keck_RMI_cdd     int64
dtype: object

Molecule                 object
Xing_MTDH-SND1_retest     int64
dtype: object

Molecule             object
Xing_MTDH-SND1_DR     int64
dtype: object


In [18]:
print Keck_Pria_Primary.dtypes
print
print Keck_RMI_cdd.dtypes
print
print Xing_MTDH_cdd.dtypes

Molecule                 object
Keck_Pria_Continuous    float64
dtype: object

Molecule                  object
FP counts % inhibition     int64
dtype: object

Molecule                      object
MTDH-SND1: FRET_decrease     float64
MTDH-SND1: CFP_increase      float64
MTDH-SND1: RATIO_decrease    float64
dtype: object


In [79]:
f = open('../dataset/data_preprocessing/lifechem123_cleaned_2017_03_10.smi', 'r')
mol_smile_dict = {}
for line in f:
    line = line.strip()
    row = line.split(' ')
    smiles = row[0]
    mol = row[1]
    mol_smile_dict[mol] = smiles

# Transform data

In [26]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives_2017_03_10.xlsx')
updated_binary_label = discrete_file.parse('Keck_Pria_Retest')
transform_data('../dataset/lc123_keckdata.sdf', '../dataset/keck_updated_complete.csv', updated_binary_label)

84075 items in all, 62608 are good, 1736 are bad


# Split Unique Data and Complement Data

In [27]:
mol_whole = updated_binary_label['Molecule'].tolist()
print len(mol_whole)

temp = pd.read_csv('../dataset/keck_updated_complete.csv')
smaller = set(temp['molecule ID(RegID)'].tolist())
print len(smaller)

output_file = open('../dataset/complement_data.csv', 'w')
i = 0
for m in mol_whole:
    if m not in smaller:
        i += 1
        print >> output_file, m
print i

72423
62608
9815


In [28]:
print len(mol_whole)

72423


In [29]:
file_name_list = ['../dataset/LC1_CDD_SD_20160818.sdf',
                  '../dataset/LC2_CDD_SD_20160818.sdf',
                  '../dataset/LC3_CDD_SD_20160818.sdf']

whole_set = set()
for file_name in file_name_list:
    suppl = Chem.SDMolSupplier(file_name)
    unique = set()
    for mol in suppl:
        if mol is None:
            continue
        name = mol.GetProp("Molecule Name")
        unique.add(name)
#         whole_set.add(name)
    whole_set = whole_set | unique
    print(len(unique))
print len(whole_set)

24992
24850
24952
72466


In [30]:
print len(mol_whole)

72423


In [31]:
i = 0
for m in whole_set:
    if m not in mol_whole:
        i += 1
        print m
print i

SMSSF-0058605
SMSSF-0548341
SMSSF-0059064
SMSSF-0060689
SMSSF-0061157
SMSSF-0053410
SMSSF-0054699
SMSSF-0061407
SMSSF-0058779
SMSSF-0061613
SMSSF-0058823
SMSSF-0041248
SMSSF-0058518
SMSSF-0042146
SMSSF-0059972
SMSSF-0063027
SMSSF-0062983
SMSSF-0548201
SMSSF-0548381
SMSSF-0060733
SMSSF-0548369
SMSSF-0046450
SMSSF-0059947
SMSSF-0548255
SMSSF-0063888
SMSSF-0060022
SMSSF-0548358
SMSSF-0041136
SMSSF-0058735
SMSSF-0060882
SMSSF-0053366
SMSSF-0053364
SMSSF-0059682
SMSSF-0058649
SMSSF-0061113
SMSSF-0058561
SMSSF-0058693
SMSSF-0054655
SMSSF-0050446
SMSSF-0062870
SMSSF-0063836
SMSSF-0053408
SMSSF-0548326
SMSSF-0053539
SMSSF-0064392
SMSSF-0548238
SMSSF-0548216
SMSSF-0046494
SMSSF-0060388
SMSSF-0548229
SMSSF-0040590
51


# Split data into k-fold

In [4]:
k = 5
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

In [5]:
k = 10
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_6_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_7_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_8_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_9_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_10_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

# Merge data from splitting folds to form training and testing

In [6]:
train_pd = read_merged_data(output_file_list[:9])
test_pd = read_merged_data([output_file_list[9]])

# Test feature- and label- extraction

In [7]:
print train_pd.dtypes

X_train, y_train = extract_feature_and_label(train_pd)
X_test, y_test = extract_feature_and_label(test_pd)

Unnamed: 0            object
molecule ID(RegID)    object
library               object
existing SMILES       object
generated SMILES      object
1024_fingerprint      object
Pria_SSB_%INH         object
true_label            object
dtype: object


# Test evaluation metrics

In [8]:
y_actual = np.eye(4)
y_pred = np.eye(4)
y_pred[2] = np.array([0, 0, 1, 1])
y_pred[3] = np.array([0, 0, 1, 0])

for i in range(4):
    print roc_auc_multi(y_actual, y_pred, [i], np.mean)
    print roc_auc_multi(y_actual, y_pred, [i], np.median)
    print enrichment_factor_multi(y_actual, y_pred, 0.5)
    print

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.833333333333
0.833333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.333333333333
0.333333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]



# Get updated data, and give some analysis

In [9]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives.xlsx')
continuous_file = pd.ExcelFile('../dataset/screening_smsf_continuous.xlsx')

print discrete_file.sheet_names
print continuous_file.sheet_names

[u'Keck_Pria_Retest', u'Keck_Pria_FP', u'Keck_RMI', u'Xing_MTDH_Retest', u'Xing_MTDH_DR']
[u'Keck_Pria_Primary', u'Keck_RMI_cdd', u'Xing_MTDH_cdd']


In [10]:
binary_data = discrete_file.parse('Keck_Pria_Retest')
print 'reset binary labels:'
print binary_data.dtypes

print

continuous_data = continuous_file.parse('Keck_Pria_Primary')
print 'cts labels:'
print continuous_data.dtypes

reset binary labels:
Molecule               object
Keck_Pria_AS_Retest     int64
dtype: object

cts labels:
Molecule                 object
Keck_Pria_Continuous    float64
dtype: object


# List all active compounds (updated version)

In [11]:
print binary_data[binary_data['Keck_Pria_AS_Retest']>0]

            Molecule  Keck_Pria_AS_Retest
576    SMSSF-0015261                    1
7315   SMSSF-0021761                    1
13763  SMSSF-0027944                    1
16138  SMSSF-0030209                    1
16803  SMSSF-0030826                    1
16837  SMSSF-0030860                    1
16901  SMSSF-0030922                    1
16907  SMSSF-0030928                    1
19073  SMSSF-0032984                    1
25686  SMSSF-0039184                    1
26232  SMSSF-0039974                    1
26511  SMSSF-0040244                    1
29310  SMSSF-0042907                    1
29965  SMSSF-0043541                    1
30010  SMSSF-0043585                    1
30569  SMSSF-0044122                    1
30603  SMSSF-0044155                    1
30614  SMSSF-0044166                    1
30661  SMSSF-0044210                    1
30755  SMSSF-0044297                    1
30756  SMSSF-0044297                    1
30888  SMSSF-0044424                    1
31008  SMSSF-0044540              